this notebook is ispired from: [Colmenero 2014](http://mseas.mit.edu/publications/Theses/Jorge_Colmenero_BS_Thesis_MIT2014.pdf)

In [ ]:
import os
import numpy as np
import hvplot.xarray
from scipy.ndimage import binary_dilation

import seareport_data as S

In [ ]:
ds = S.gebco_ds("sub_ice")
ds

In [ ]:
subset = ds.sel(lon=slice(-72,-62), lat=slice(40, 46))
subset.elevation.shape
subset.elevation.hvplot.image(cmap="cet_CET_R4").opts(width = 600, height=500)

In [ ]:
import tqdm 
def _earth_gradient(F, dx, dy):
    """
    earth_gradient(F,HX,HY), where F is 2-D, uses the spacing
    specified by HX and HY. HX and HY can either be scalars to specify
    the spacing between coordinates or vectors to specify the
    coordinates of the points.  If HX and HY are vectors, their length
    must match the corresponding dimension of F.
    """
    Fy, Fx = np.zeros(F.shape), np.zeros(F.shape)

    # Forward diferences on edges
    Fx[:, 0] = (F[:, 1] - F[:, 0]) / dx
    Fx[:, -1] = (F[:, -1] - F[:, -2]) / dx
    Fy[0, :] = (F[1, :] - F[0, :]) / dy
    Fy[-1, :] = (F[-1, :] - F[-2, :]) / dy

    # Central Differences on interior
    Fx[:, 1:-1] = (F[:, 2:] - F[:, :-2]) / (2 * dx)
    Fy[1:-1, :] = (F[2:, :] - F[:-2, :]) / (2 * dy)

    return Fy, Fx


def calc_gradient(ds):
    elevation = ds.elevation.values
    lon = ds.lon.values
    lat = ds.lat.values

    dx = np.diff(lon)[0]
    dy = np.diff(lat)[0]

    mean_latitude = np.mean(lat)
    meters_per_degree = (
            111132.92
            - 559.82 * np.cos(2 * mean_latitude)
            + 1.175 * np.cos(4 * mean_latitude)
            - 0.0023 * np.cos(6 * mean_latitude)
        )
    dy *= meters_per_degree
    dx *= meters_per_degree
    Fy, Fx = _earth_gradient(elevation, dx, dy)

    grad_magnitude = np.sqrt(Fy**2 + Fx**2) * meters_per_degree
    grad_direction = np.arctan2(Fy, Fx) * 180 / np.pi

    return grad_magnitude, grad_direction

def grad_smoothing(grad, threshold = 50, iterations = 100, ):
    def ingrid_smooth(g):
        return 1/4 * g + \
               1/8 * (np.roll(g, 1, axis=0) + np.roll(g, -1, axis=0) + np.roll(g, 1, axis=1) + np.roll(g, -1, axis=1)) + \
               1/16 * (np.roll(np.roll(g, 1, axis=0), 1, axis=1) + np.roll(np.roll(g, 1, axis=0), -1, axis=1) + \
                       np.roll(np.roll(g, -1, axis=0), -1, axis=1) + np.roll(np.roll(g, -1, axis=0), 1, axis=1))
    def left_edge_smooth(g):
        return 3/8 * g + \
               1/8 * (np.roll(g, 1, axis=0) + np.roll(np.roll(g, 1, axis=0), 1, axis=1) + np.roll(np.roll(g, 1, axis=0), -1, axis=1) + \
                       np.roll(g, 1, axis=1) + np.roll(g, -1, axis=1))

    def right_edge_smooth(g):
        return 3/8 * g + \
               1/8 * (np.roll(g, -1, axis=0) + np.roll(np.roll(g, -1, axis=0), 1, axis=1) + np.roll(np.roll(g, -1, axis=0), -1, axis=1) + \
                       np.roll(g, 1, axis=1) + np.roll(g, -1, axis=1))

    def top_edge_smooth(g):
        return 3/8 * g + \
               1/8 * (np.roll(g, -1, axis=0) + np.roll(np.roll(g, -1, axis=0), 1, axis=1) + np.roll(np.roll(g, -1, axis=0), -1, axis=1) + \
                       np.roll(g, 1, axis=1) + np.roll(g, -1, axis=1))

    def bottom_edge_smooth(g):
        return 3/8 * g + \
               1/8 * (np.roll(g, 1, axis=0) + np.roll(g, -1, axis=0) + np.roll(np.roll(g, 1, axis=0), -1, axis=1) + \
                       np.roll(np.roll(g, -1, axis=0), -1, axis=1) + np.roll(g, -1, axis=1))

    def upper_left_corner_smooth(g):
        return 1/3 * g + \
               2/9 * (np.roll(g, 1, axis=0) + np.roll(np.roll(g, 1, axis=0), 1, axis=1) + np.roll(g, 1, axis=1))

    def upper_right_corner_smooth(g):
        return 1/3 * g + \
               2/9 * (np.roll(g, 1, axis=0) + np.roll(np.roll(g, 1, axis=0), -1, axis=1) + np.roll(g, -1, axis=1))

    def lower_left_corner_smooth(g):
        return 1/3 * g + \
               2/9 * (np.roll(g, -1, axis=0) + np.roll(np.roll(g, -1, axis=0), 1, axis=1) + np.roll(g, 1, axis=1))

    def lower_right_corner_smooth(g):
        return 1/3 * g + \
               2/9 * (np.roll(g, -1, axis=0) + np.roll(np.roll(g, -1, axis=0), -1, axis=1) + np.roll(g, -1, axis=1))

    smoothed = grad.copy()

    # Create masks for different regions
    in_grid_mask = np.zeros_like(grad, dtype=bool)
    left_edge_mask = np.zeros_like(grad, dtype=bool)
    right_edge_mask = np.zeros_like(grad, dtype=bool)
    top_edge_mask = np.zeros_like(grad, dtype=bool)
    bottom_edge_mask = np.zeros_like(grad, dtype=bool)
    upper_left_corner_mask = np.zeros_like(grad, dtype=bool)
    upper_right_corner_mask = np.zeros_like(grad, dtype=bool)
    lower_left_corner_mask = np.zeros_like(grad, dtype=bool)
    lower_right_corner_mask = np.zeros_like(grad, dtype=bool)

    # Fill the masks
    in_grid_mask[1:-1, 1:-1] = True
    left_edge_mask[0, 1:-1] = True
    right_edge_mask[-1, 1:-1] = True
    top_edge_mask[1:-1, 0] = True
    bottom_edge_mask[1:-1, -1] = True
    upper_left_corner_mask[0, 0] = True
    upper_right_corner_mask[0, -1] = True
    lower_left_corner_mask[-1, 0] = True
    lower_right_corner_mask[-1, -1] = True

    # Apply threshold
    for it in tqdm.tqdm(range(iterations)):
        
        threshold_mask = smoothed > threshold if threshold > 0 else smoothed < threshold
        threshold_mask = binary_dilation(threshold_mask, iterations=1)

        # Combine masks with threshold
        in_grid_mask &= threshold_mask
        left_edge_mask &= threshold_mask
        right_edge_mask &= threshold_mask
        top_edge_mask &= threshold_mask
        bottom_edge_mask &= threshold_mask
        upper_left_corner_mask &= threshold_mask
        upper_right_corner_mask &= threshold_mask
        lower_left_corner_mask &= threshold_mask
        lower_right_corner_mask &= threshold_mask

        # Iterate over each node in the elevation data
        smoothed[in_grid_mask] = ingrid_smooth(smoothed)[in_grid_mask]
        smoothed[left_edge_mask] = left_edge_smooth(smoothed)[left_edge_mask]
        smoothed[right_edge_mask] = right_edge_smooth(smoothed)[right_edge_mask]
        smoothed[top_edge_mask] = top_edge_smooth(smoothed)[top_edge_mask]
        smoothed[bottom_edge_mask] = bottom_edge_smooth(smoothed)[bottom_edge_mask]
        smoothed[upper_left_corner_mask] = upper_left_corner_smooth(smoothed)[upper_left_corner_mask]
        smoothed[upper_right_corner_mask] = upper_right_corner_smooth(smoothed)[upper_right_corner_mask]
        smoothed[lower_left_corner_mask] = lower_left_corner_smooth(smoothed)[lower_left_corner_mask]
        smoothed[lower_right_corner_mask] = lower_right_corner_smooth(smoothed)[lower_right_corner_mask]
    
    threshold_mask = smoothed > threshold if threshold > 0 else smoothed < threshold
    smoothed[threshold_mask]=threshold
    return smoothed

def laplacian_smothing(grad, threshold=50, iterations=100):
    smoothed = grad.copy()
    def laplacian_smooth(g, threshold):
        grad_x = np.gradient(g, axis=1)
        grad_y = np.gradient(g, axis=0)
        grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        if threshold >0:
            scale_factor = np.where(g > threshold, threshold / grad_magnitude, 1.0)
        else: 
            scale_factor = np.where(g < threshold, threshold / grad_magnitude, 1.0)
        grad_x_rescaled = grad_x * scale_factor
        grad_y_rescaled = grad_y * scale_factor

        laplacian = (
            np.gradient(grad_x_rescaled, axis=1) + np.gradient(grad_y_rescaled, axis=0)
        )
        return g + laplacian

    for iteration in tqdm.tqdm(range(iterations)):
        smoothed = laplacian_smooth(smoothed, threshold)
        
    if threshold > 0: 
        threshold_mask = smoothed > threshold 
        smoothed[threshold_mask]=threshold
        zero_mask = smoothed < 0 
        smoothed[zero_mask] = 0
    else: 
        threshold_mask = smoothed < threshold 
        smoothed[threshold_mask]=threshold
        zero_mask = smoothed > 0 
        smoothed[zero_mask] =  0
    return smoothed

In [ ]:
grad, dir = calc_gradient(subset)

subset["grad"] = (("lat", "lon"), grad)
subset["dir"] = (("lat", "lon"), dir)
subset

In [ ]:
gmax = 10000
smoothed =  grad_smoothing(subset.grad.data, gmax, iterations = 10)

subset["smoothed"] = (("lat", "lon"), smoothed)
subset.smoothed.hvplot.image(cmap="cet_CET_R4").opts(width=600, height=500,clim = (0, gmax))
subset.smoothed.hvplot.hist()

In [ ]:
gmax = 10000
smoothed =  laplacian_smothing(subset.grad.data, gmax, iterations = 3)

subset["smoothed"] = (("lat", "lon"), smoothed)
subset.smoothed.hvplot.image(cmap="cet_CET_R4").opts(width=600, height=500,clim = (0, gmax))
subset.smoothed.hvplot.hist()

reduced gradient adaptation

In [ ]:
elevation = subset.elevation.data
elevation[elevation > -10 ]= -10
grad, dir = calc_gradient(subset)

subset["reduced_grad"] = (("lat", "lon"), grad/elevation)

In [ ]:
rgmax = -20
rg_smoothed =  grad_smoothing(subset.reduced_grad.data, rgmax, iterations = 20)

subset["reduced_grad_smooth"] = (("lat", "lon"), rg_smoothed)
subset.reduced_grad_smooth.hvplot.image(cmap="cet_CET_R4").opts(width=600, height=500)
subset.reduced_grad_smooth.hvplot.hist()

In [ ]:
rgmax = -20
rg_smoothed =  laplacian_smothing(subset.reduced_grad.data, rgmax, iterations = 20)

subset["reduced_grad_smooth"] = (("lat", "lon"), rg_smoothed)
subset.reduced_grad_smooth.hvplot.image(cmap="cet_CET_R4").opts(width=600, height=500)
subset.reduced_grad_smooth.hvplot.hist()

mesh size contributions

In [ ]:
def h_size_grad(values, min_elem_size, max_elem_size):
    return max_elem_size - (values/np.max(values)*(max_elem_size-min_elem_size))
def h_size_rgrad(values, min_elem_size, max_elem_size):
    return max_elem_size - (values/np.min(values)*(max_elem_size-min_elem_size))


In [ ]:
gmax = 10000
grad, dir = calc_gradient(subset)
smoothed =  grad_smoothing(grad, gmax, iterations = 20)
rgmax = -20
rg_smoothed =  grad_smoothing(grad/elevation, rgmax, iterations = 20)

In [ ]:
h_grad = h_size_grad(smoothed, 10, 1000)
h_rg = h_size_rgrad(rg_smoothed, 10, 1000)

h = np.minimum(h_grad, h_rg)
subset["h"] = (("lat", "lon"), h)
# subset.h.hvplot.image(cmap="cet_CET_R4").opts(width=600, height=500)
# subset.h.hvplot.hist()